In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd
from matplotlib import cm
from matplotlib.patches import Circle
import seaborn as sns
from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# -------------------------------------------------------------------
# Plot Styling
# -------------------------------------------------------------------
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

unknown_country_codes = []

def load_port_data(csv_filepath):
    """Load and prepare port data from CSV."""
    df = pd.read_csv(csv_filepath)
    df_ports = df.dropna(subset=['Latitude', 'Longitude']).copy()
    return df_ports

def create_world_map(df_ports, save_path='ports_global.png', dpi=300):
    """Create a global map showing all port locations with country contours."""
    fig = plt.figure(figsize=(20, 10))
    ax = plt.axes(projection='rectilinear')
    
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    
    world.plot(ax=ax, 
              color='#f0f0f0',
              edgecolor='#666666',
              linewidth=0.8,
              alpha=0.5)
    
    ax.set_facecolor('#e6f2ff')
    
    ax.set_xlim(-180, 180)
    ax.set_ylim(-90, 90)
    
    ax.grid(True, alpha=0.2, linestyle='--', color='gray')
    
    region_colors = {
        'EU': '#3498db',  # Blue -> Europe
        'AS': '#e74c3c',  # Red -> Asia
        'NA': '#2ecc71',  # Green -> North America
        'SA': '#f39c12',  # Orange -> South America
        'AF': '#9b59b6',  # Purple -> Africa
        'OC': '#1abc9c',  # Turquoise -> Oceania
    }
    
    unknown_country_codes = []
    
    def get_region(country_code):
        
        # -------------------------------------------------------------------
        # Europe Country Codes
        # -------------------------------------------------------------------
        europe = ['AD', 'AL', 'AT', 'BA', 'BE', 'BG', 'BY', 'CH', 'CY', 'CZ', 'DE', 
                  'DK', 'EE', 'ES', 'FI', 'FR', 'GB', 'GR', 'HR', 'HU', 'IE', 'IS', 
                  'IT', 'LT', 'LU', 'LV', 'MC', 'MD', 'ME', 'MK', 'MT', 'NL', 'NO', 
                  'PL', 'PT', 'RO', 'RS', 'RU', 'SE', 'SI', 'SK', 'SM', 'UA', 'VA']
        
        # -------------------------------------------------------------------
        # Asia Country Codes
        # -------------------------------------------------------------------
        asia = ['AE', 'AF', 'AM', 'AZ', 'BD', 'BH', 'BN', 'BT', 'CN', 'GE', 'HK', 
                'ID', 'IL', 'IN', 'IQ', 'IR', 'JO', 'JP', 'KG', 'KH', 'KP', 'KR', 
                'KW', 'KZ', 'LA', 'LB', 'LK', 'MM', 'MN', 'MO', 'MV', 'MY', 'NP', 
                'OM', 'PH', 'PK', 'PS', 'QA', 'SA', 'SG', 'SY', 'TH', 'TJ', 'TL', 
                'TM', 'TR', 'TW', 'UZ', 'VN', 'YE']
        
        # -------------------------------------------------------------------
        # North America Country Codes
        # -------------------------------------------------------------------
        north_america = ['AG', 'AI', 'AW', 'BB', 'BL', 'BM', 'BQ', 'BS', 'BZ', 'CA', 
                         'CR', 'CU', 'CW', 'DM', 'DO', 'GD', 'GL', 'GP', 'GT', 'HN', 
                         'HT', 'JM', 'KN', 'KY', 'LC', 'MF', 'MQ', 'MS', 'MX', 'NI', 
                         'PA', 'PM', 'PR', 'SV', 'SX', 'TC', 'TT', 'US', 'VC', 'VG', 
                         'VI']
        
        # -------------------------------------------------------------------
        # South America Country Codes
        # -------------------------------------------------------------------
        south_america = ['AR', 'BO', 'BR', 'CL', 'CO', 'EC', 'FK', 'GF', 'GY', 'PE', 
                         'PY', 'SR', 'UY', 'VE']
        
        # -------------------------------------------------------------------
        # Africa Country Codes
        # -------------------------------------------------------------------
        africa = ['AO', 'BF', 'BI', 'BJ', 'BW', 'CD', 'CF', 'CG', 'CI', 'CM', 'CV', 
                  'DJ', 'DZ', 'EG', 'EH', 'ER', 'ET', 'GA', 'GH', 'GM', 'GN', 'GQ', 
                  'GW', 'KE', 'KM', 'LR', 'LS', 'LY', 'MA', 'MG', 'ML', 'MR', 'MU', 
                  'MW', 'MZ', 'NA', 'NE', 'NG', 'RE', 'RW', 'SC', 'SD', 'SL', 'SN', 
                  'SO', 'SS', 'ST', 'SZ', 'TD', 'TG', 'TN', 'TZ', 'UG', 'YT', 'ZA', 
                  'ZM', 'ZW']
        
        # -------------------------------------------------------------------
        # Oceania Country Codes
        # -------------------------------------------------------------------
        oceania = ['AS', 'AU', 'CC', 'CK', 'CX', 'FJ', 'FM', 'GU', 'KI', 'MH', 'MP', 
                   'NC', 'NF', 'NR', 'NU', 'NZ', 'PF', 'PG', 'PN', 'PW', 'SB', 'TK', 
                   'TO', 'TV', 'UM', 'VU', 'WF', 'WS']
        
        if country_code in europe:
            return 'EU'
        elif country_code in asia:
            return 'AS'
        elif country_code in north_america:
            return 'NA'
        elif country_code in south_america:
            return 'SA'
        elif country_code in africa:
            return 'AF'
        elif country_code in oceania:
            return 'OC'
        else:
            if country_code not in unknown_country_codes:
                unknown_country_codes.append(country_code)
                print(f"Unknown country code: {country_code}")
            return 'OT'
    
    # -------------------------------------------------------------------
    # Plot Ports by Region
    # -------------------------------------------------------------------
    for region, color in region_colors.items():
        region_ports = df_ports[df_ports['Country'].apply(get_region) == region]
        if len(region_ports) > 0:
            ax.scatter(region_ports['Longitude'], 
                       region_ports['Latitude'], 
                       c=color, 
                       s=20,
                       alpha=0.6,
                       edgecolors='white', 
                       linewidth=0.5,
                       zorder=5
            )
            
    plt.axis('off')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=dpi, bbox_inches='tight', facecolor='white')
    plt.close()
    
    print(f"Map saved to: {save_path}")

In [ ]:
csv_filepath = "../data/port_data/unlocode/unlocode_ports_only_20250604_144152.csv"
output_directory = "../output/port_maps"
os.makedirs(output_directory, exist_ok=True)
df_ports = load_port_data(csv_filepath)
create_world_map(df_ports, os.path.join(output_directory, "ports_global.png"), dpi=300)